In [2]:
import self_made_functions as smf 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from sklearn.tree import DecisionTreeClassifier

df_assesment, _ = smf.get_df_correct()

# Make the input used for train the classefier model
input_assesmen = pd.DataFrame(columns=['CER', 'Score'])

same_score = pd.DataFrame(columns=['pronScores', 'Score'])

for idx, row in df_assesment.iterrows():
    pronScore = row['pronScores'].split(' ')
    CER = pronScore.count('0')/len(pronScore)
    
    one = pronScore.count('1')
    values_one = {
        'pronScore': one,
        'Score': row['Score']
    }
    values_one = pd.DataFrame(values_one, index=[0])
    same_score = pd.concat([same_score, values_one], ignore_index=True)
    
    
    values = {
            'CER': CER,
            'Score': row['Score']
        }
    values = pd.DataFrame(values, index=[0])
    # if values.empty: No emty enteries
    #     print('empty', idx)
    # elif input_assesmen.empty:
    #     print('empty', idx)
        
    input_assesmen = pd.concat([input_assesmen, values], ignore_index=True)

/tmp/ipykernel_3312879/938844425.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  input_assesmen = pd.concat([input_assesmen, values], ignore_index=True)


In [ ]:
X = input_assesmen[['CER']]
y = input_assesmen['Score'].astype(int) 

# Split data
# Set random state for reproducibility (same results every time we run the code)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


df_assesment

In [3]:

import torch
import librosa
from transformers import AutoTokenizer, AutoModelWithLMHead

# Load the Whisper tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("openai/whisper-small")
model = AutoModelWithLMHead.from_pretrained("openai/whisper-small")

def map_to_array(batch):
    speech, _ = librosa.load(batch["file"], sr=16000, mono=True)
    batch["speech"] = speech
    return batch

def map_to_pred_whisper(batch):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_values = tokenizer(batch["speech"], return_tensors="pt", padding="max_length", truncation=True).input_values
    input_values = input_values.to(device)
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.decode(predicted_ids.squeeze(), skip_special_tokens=True)
    batch["transcription"] = transcription
    batch['predictions'] = predicted_ids
    return batch

# Assuming `test_dataset` is your dataset object

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/ajtruyen/anaconda3/envs/master_amanda/lib/python3.12/site-packages/transformers/models/auto/modeling_auto.py:1712: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [5]:
test_dataset = df_assesment.map(map_to_array)
result = df_assesment.map(map_to_pred_whisper)

TypeError: string indices must be integers, not 'str'